In [37]:
from brightway2 import *
from bw2regional.ecoinvent import *
import xlrd

In [2]:
projects.current = "Better World"

In [3]:
bw2setup()

Biosphere database already present!!! No setup is needed


In [4]:
ei = SingleOutputEcospold2Importer(
    "/Users/cmutel/Documents/LCA Documents/Ecoinvent/3.1/cutoff/datasets", 
    "ecoinvent 3.1 cutoff"
)
ei.apply_strategies()
ei.write_database()
ei = None

Extracting ecospold2 files:
0%                          100%
[##############################] | ETA[sec]: 0.000 | Item ID: fff527b1-0fe4-4
Total time elapsed: 105.502 sec


Title: Extracting ecospold2 files:
  Started: 10/06/2015 19:48:54
  Finished: 10/06/2015 19:50:39
  Total time elapsed: 105.502 sec
  CPU %: 93.200000
  Memory %: 12.428045
Extracted 11301 datasets in 105.62 seconds
Applying strategy: normalize_units
Applying strategy: remove_zero_amount_coproducts
Applying strategy: remove_zero_amount_inputs_with_no_activity
Applying strategy: remove_unnamed_parameters
Applying strategy: es2_assign_only_product_with_amount_as_reference_product
Applying strategy: assign_single_product_as_activity
Applying strategy: create_composite_code
Applying strategy: drop_unspecified_subcategories
Applying strategy: link_biosphere_by_flow_uuid
Applying strategy: link_internal_technosphere_by_composite_code
Applying strategy: delete_exchanges_missing_activity
Applying strategy: delete_ghost_exchanges
Applied 12 strategies in 3.56 seconds


How many datasets?

In [90]:
db = Database("ecoinvent 3.1 cutoff")
len(db)

11301

How many global datasets?

In [87]:
len([1 for x in db if x['location'] == 'GLO'])

3187

How many RoW datasets?

In [89]:
len([1 for x in db if x['location'] == 'RoW'])

2683

How many distinct Rest of Worlds?

In [72]:
_, row_locations, _ = discretize_rest_of_world("ecoinvent 3.1 cutoff", warn=False)
len(row_locations)

137

Get disaggregation data

In [31]:
wb = xlrd.open_workbook("files/exiobase-extract.xlsx")

In [32]:
sheet = wb.sheet_by_name("Products")
products = {sheet.cell(row, 0).value: sheet.cell(row, 1).value for row in range(1, sheet.nrows)}

In [33]:
sheet = wb.sheet_by_name("Codes")
codes = {
    int(sheet.cell(row, 1).value): [
        'p' + obj[1:] 
        for obj in sheet.cell(row, 2).value.split(";")
    ] 
    for row in range(2, sheet.nrows)
}

In [30]:
codes = {key: [products[obj] for obj in value if obj in products] for key, value in codes.items()}
codes

{1: ['Paddy rice',
  'Wheat',
  'Cereal grains nec',
  'Vegetables, fruit, nuts',
  'Oil seeds',
  'Sugar cane, sugar beet',
  'Plant-based fibers',
  'Crops nec',
  'Cattle',
  'Pigs',
  'Poultry',
  'Meat animals nec',
  'Animal products nec',
  'Raw milk',
  'Wool, silk-worm cocoons',
  'Manure (conventional treatment)',
  'Manure (biogas treatment)'],
 2: ['Products of forestry, logging and related services'],
 5: ['Fish and other fishing products; services incidental of fishing'],
 10: [],
 11: ['Crude petroleum and services related to crude oil extraction, excluding surveying',
  'Other Hydrocarbons'],
 12: ['Uranium and thorium ores'],
 13: ['Paddy rice',
  'Wheat',
  'Cereal grains nec',
  'Vegetables, fruit, nuts',
  'Oil seeds',
  'Sugar cane, sugar beet',
  'Plant-based fibers',
  'Crops nec',
  'Cattle',
  'Pigs',
  'Poultry',
  'Meat animals nec',
  'Animal products nec',
  'Raw milk',
  'Wool, silk-worm cocoons',
  'Manure (conventional treatment)',
  'Manure (biogas trea

In [36]:
sheet = wb.sheet_by_name("Physical supply 2007")
supply = [(
        sheet.cell(row, 0).value,
        sheet.cell(row, col).value,
        sheet.cell(0, col).value
) 
for row in range(1, sheet.nrows)
for col in range(2, sheet.ncols)
]
supply

[('Paddy rice', 0.0, 'Austria'),
 ('Paddy rice', 0.0, 'Belgium'),
 ('Paddy rice', 19580.3406992209, 'Bulgaria'),
 ('Paddy rice', 0.0, 'Cyprus'),
 ('Paddy rice', 0.0, 'Czech Republic'),
 ('Paddy rice', 0.0, 'Germany'),
 ('Paddy rice', 0.0, 'Denmark'),
 ('Paddy rice', 0.0, 'Estonia'),
 ('Paddy rice', 1478479.46274253, 'Spain'),
 ('Paddy rice', 0.0, 'Finland'),
 ('Paddy rice', 352904.77975731, 'France'),
 ('Paddy rice', 216091.876492565, 'Greece'),
 ('Paddy rice', 7959.87224111294, 'Hungary'),
 ('Paddy rice', 0.0, 'Ireland'),
 ('Paddy rice', 1112715.025, 'Italy'),
 ('Paddy rice', 0.0, 'Lithuania'),
 ('Paddy rice', 0.0, 'Luxembourg'),
 ('Paddy rice', 0.0, 'Latvia'),
 ('Paddy rice', 0.0, 'Malta'),
 ('Paddy rice', 0.0, 'Netherlands'),
 ('Paddy rice', 0.0, 'Poland'),
 ('Paddy rice', 447393.241025738, 'Portugal'),
 ('Paddy rice', 23413.7594349601, 'Romania'),
 ('Paddy rice', 0.0, 'Sweden'),
 ('Paddy rice', 0.0, 'Slovenia'),
 ('Paddy rice', 0.0, 'Slovakia'),
 ('Paddy rice', 0.0, 'United Kingdom

Get spatial scale for exiobase

In [93]:
exiobase_world = {x[2] for x in supply}
for x in sorted(exiobase_world):
    print(x)

Australia
Austria
Belgium
Brazil
Bulgaria
Canada
China
Cyprus
Czech Republic
Denmark
Estonia
Finland
France
Germany
Greece
Hungary
India
Indonesia
Ireland
Italy
Japan
Latvia
Lithuania
Luxembourg
Malta
Mexico
Netherlands
Norway
Poland
Portugal
RoW Africa
RoW America
RoW Asia and Pacific
RoW Europe
RoW Middle East
Romania
Russia
Slovakia
Slovenia
South Africa
South Korea
Spain
Sweden
Switzerland
Taiwan
Turkey
United Kingdom
United States


Get datasets that can be disaggregated, based on industry code matching with what is available in exiobase

In [59]:
def get_code_in_exiobase_codes(code):
    code = code.split(":")[0]
    # Take care of 1000a codes - strip last letter
    try:
        int(code)
    except ValueError:
        code = code[:-1]
    if int(code) in codes:
        return int(code)
    elif len(code) > 2 and int(code[:-2]) in codes:
        return int(code[:-2])
    elif len(code) > 4 and int(code[:-4]) in codes:
        return int(code[:-4])
    else:
        return None

In [94]:
to_disaggregate = set()

for ds in db:
    if ds['location'] == 'GLO' and ds.get('classifications'):
        try:
            code = get_code_in_exiobase_codes(dict(ds['classifications'])['ISIC rev.4 ecoinvent'])
            assert code
            to_disaggregate.add((code, ds))
        except (KeyError, AssertionError):
            continue

In [95]:
print(len(to_disaggregate), "out of 2683 can be disaggregated")

2398 out of 2683 can be disaggregated


In [96]:
to_disaggregate

{(2, 'market for cork slab' (kilogram, GLO, None)),
 (2732, 'cable production, three-conductor cable' (meter, GLO, None)),
 (26, 'market for computer, desktop, without screen' (unit, GLO, None)),
 (16,
  'market for wood preservation, spray tunnel/deluging, water-based, outdoor use, no ground contact' (kilogram, GLO, None)),
 (2021, 'market for benzimidazole-compound' (kilogram, GLO, None)),
 (2710, 'market for transformer, high voltage use' (kilogram, GLO, None)),
 (30, 'market for offshore platform, natural gas' (unit, GLO, None)),
 (1, 'market for grass silage, organic' (kilogram, GLO, None)),
 (10, 'palm oil refinery operation' (kilogram, GLO, None)),
 (2610,
  'cable production, network cable, category 5, without plugs' (meter, GLO, None)),
 (24, 'market for precious metal for jewellery' (cubic meter, GLO, None)),
 (20, 'market for 3-methyl-1-butanol' (kilogram, GLO, None)),
 (25,
  'market for impact extrusion of steel, hot, 2 strokes' (kilogram, GLO, None)),
 (2610,
  'market fo

Get statistics on what level the industry codes are:

In [99]:
(
    len([1 for x in to_disaggregate if len(str(x[0])) in (1, 2)]),
    len([1 for x in to_disaggregate if len(str(x[0])) in (3,4)]),
    len([1 for x in to_disaggregate if len(str(x[0])) > 4])
)

(1715, 683, 0)

In [84]:
replacable_exchanges = 0
fixable_keys = {x.key for x in to_disaggregate}
specific_exchanges = 0

switchable_locations = {"AU","BE","BG","CY","CZ","DK","ES","FI","FR","DE","GR","HU",
                        "IR","IT","LV","LT","LU","MT","NL","PL","PT","RO","SI","SK",
                        "ES","SE","UK","AT","BR","CA","CN","IN","ID","JP","NO","MX",
                        "RU","SA","KR","CH","TW","TR","US"}

for ds in to_disaggregate:
    groups = []
    for exc in ds.technosphere():
        if exc.input.key in fixable_keys:
            replacable_exchanges += 1
            continue
        else:
            groups.append(exc.input)
    container = {}
    for ds in groups:
        container.setdefault(ds['name'], []).append(ds)
    for group in container.values():
        if any(obj for obj in group if obj['location'] in switchable_locations):
            specific_exchanges += 1

In [70]:
replacable_exchanges

4940

In [85]:
specific_exchanges

2453